In [0]:
#import shutil

#shutil.rmtree('./flower_data')


In [1]:
!pip install torch
!pip install torchvision
!pip install requests

    100% |████████████████████████████████| 591.8MB 23kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x60dce000 @  0x7f62a3fd22a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 2.4MB/s 
    100% |████████████████████████████████| 2.0MB 14.1MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [2]:
import os
import torch
import torchvision
from torchvision import transforms, datasets
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
#import 
print('Done')

Done


In [21]:
data_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.CenterCrop(500),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.4406], std=[0.229,0.224,0.225])
])

data_transform

Compose(
    RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
    CenterCrop(size=(500, 500))
    RandomHorizontalFlip(p=0.5)
    Resize(size=224, interpolation=PIL.Image.BILINEAR)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.4406], std=[0.229, 0.224, 0.225])
)

In [4]:
import requests, zipfile, io

r = requests.get('https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

print('Done')

Done


In [0]:
data_dir = './flower_data/'
train_dir = os.path.join(data_dir, 'train/') 
test_dir = os.path.join(data_dir, 'valid/') 

In [0]:
# Loading images and creating labels

train_data = datasets.ImageFolder(train_dir, transform = data_transform) 
test_data = datasets.ImageFolder(test_dir, transform = data_transform) 

In [24]:
print('No of training images', len(train_data))
print('No of testing images', len(test_data))

No of training images 6552
No of testing images 818


In [0]:
# Load the data

train_loader = torch.utils.data.DataLoader(train_data, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=True)

In [0]:
train_loader

In [0]:
# Load pre-train model

#model = torchvision.models.resnet18(pretrained=True)
model = torchvision.models.resnet152(pretrained=True)


In [0]:
# Reset Gradient

for param in model.parameters():
  param.requires_grad = False  

In [0]:

# getting the no.of neurons in last layers

num_features = model.fc.in_features
num_features

torch.save(model.state_dict(), './pr_model.pt')

In [0]:
class PSC(nn.Module):
  def __init__(self):
    super(PSC, self).__init__()
    self.fc1 = nn.Linear(in_features = 2048, out_features=256)
    self.dropout = nn.Dropout(0.25)
    self.fc2 = nn.Linear(in_features = 256, out_features=64)
    self.fc3 = nn.Linear(in_features = 64, out_features=256)
    self.fc4 = nn.Linear(in_features = 256, out_features=102)
    
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.dropout(x)
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return x
    

In [0]:
model.fc = PSC()

model.load_state_dict(torch.load('pr_model.pt'), strict=False) #SFSAver10

In [0]:
#model.optimizer

In [0]:
# Define Loss and Optimizer

#loss_fn = torch.nn.CrossEntropyLoss()
loss_fn = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


In [0]:
'''
examples = enumerate(train_loader)
for (a,b) in examples:
  print(a)
  
'''

'\nexamples = enumerate(train_loader)\nfor (a,b) in examples:\n  print(a)\n  \n'

In [0]:
!pip install Pillow==4.0.0
!pip install PIL
!pip install image

#from PIL import Im

    100% |████████████████████████████████| 5.6MB 5.4MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


In [0]:
# Training our model

n_epochs=100
accuracy_val = 0.0

for epoch in range(1, n_epochs+1):
  train_loss = 0.0
  
  for batch_i, (data, target) in enumerate(train_loader):
    
    if torch.cuda.is_available():
      data, target, model = data.cuda(), target.cuda(), model.cuda()
              
    output = model(data)
    loss = loss_fn(output, target)
    train_loss +=loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
   
    
    if batch_i % 20 == 15:
      print("Epoch: {}  Batch: {} loss: {}".format(epoch, batch_i, train_loss/20))
      
      train_loss = 0.0
      model.eval()
      total_correct = 0
      total = 0
      max_avg_output = torch.argmax(output, dim=1)
      
      for number in range(len(data)):
        correct = max_avg_output[number] == target[number]
        total_correct += torch.Tensor.cpu(correct).numpy()
        total += 1
        
      print('Correct: ', total_correct)
      print('Total: ', total)
      acc = (total_correct/total)
      print('Training Accuracy: ', acc)
      
      if acc > accuracy_val:
        torch.save(model, 'PyTorch_model.pt')
        accuracy_val = acc
        
      model.train()
      
        
      




In [19]:
model1 = torch.load('PyTorch_model.pt')


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [0]:
def load_checkpoint(checkpoint_path):
  checkpoint = torch.load(checkpoint_path)
  model.load_state_dict(checkpoint['state_dict'])
  return model



In [18]:
'''
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')
        

save_checkpoint()




'''

# ... after training, save your model 
# model.save_state_dict('model_state.pt')

# .. to load your previously training model:
#model.load_state_dict(torch.load('PyTorch_model.pt'))


AttributeError: ignored

In [0]:
from collections import namedtuple
from torch import nn


def test_model(model1, criterion=nn.NLLLoss, image_size=224):
    Tester = namedtuple('Tester', ['model1', 'criterion', 'image_size'])
    return Tester(model1, criterion, image_size)
  
print(test_model(model1, criterion=nn.NLLLoss, image_size=224))

In [0]:
import torch
import sys
from torchvision import datasets, transforms, models
from torch import nn, optim
from collections import OrderedDict

def create_loaders(data_dir):
  

    train_dir = data_dir + '/train'
    valid_dir = data_dir + '/valid'
    test_dir = data_dir + '/test'
    
    norm_mean = [0.485, 0.456, 0.406]
    norm_std = [0.229, 0.224, 0.225]
    
    #Define data transforms
    data_transforms = transforms.Compose([transforms.Resize(255),
                                         transforms.CenterCrop(224),
                                         transforms.ToTensor(),
                                         transforms.Normalize(norm_mean,
                                                             norm_std)])
    #Define training data transforms
    data_trans_train = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomRotation(30),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize(norm_mean,
                                                          norm_std)])
    #Load the datasets
    train_dataset = datasets.ImageFolder(train_dir, transform=data_trans_train)
    test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms)
    valid_dataset = datasets.ImageFolder(valid_dir, transform=data_transforms)
    
    #Define dataloaders
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
    testloader = torch.utils.data.DataLoader(test_dataset, batch_size=32)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=32)
    
    #Grab class ids
    class_idx = train_dataset.class_to_idx
    
    return trainloader, testloader, validloader, class_idx

def prep_model(arch):
    """
    Selects, downloads and returns model provided. Returns model architecture for the CNN and
    the associated input_size.
    Parameters:
        arch - Used to select which architecture to use for prepare
    Returns:
        model_select[arch] - selects the variable out of a dictionary and returns the
            model associated with arch
        input_size[arch] - selects the associated input size for the model selected
            with arch
    """
    vgg16=''
    alexnet=''
    densenet121=''
    #Only download the model you need, kill program if one of the three models isn't passed
    if arch == 'vgg':
        vgg16 = models.vgg16(pretrained=True)
    elif arch == 'alexnet':
        alexnet = models.alexnet(pretrained=True)
    elif arch == 'densenet':
        densenet121 = models.densenet121(pretrained=True)
    else:
        print('{} architecture not recognized. Supported args: \'vgg\', \'alexnet\', or \'densenet\''.format(arch))
        sys.exit()
    
    #Pass the model, and grab the input size
    model_select = {'vgg':vgg16,'alexnet':alexnet,'densenet':densenet121}
    input_size = {'vgg':25088,'alexnet':9216,'densenet':1024}
    return model_select[arch], input_size[arch]

def create_classifier(model, input_size, hidden_layers, output_size, learning_rate, drop_p=0.5):
    """
    Takes a pretrained CNN, freezes the features and creates a untrained classifier. Returns
    model with an untrained classifier, loss function critierion (NLLLoss) and Adam optimizer.
    Parameters:
        model - Pretrained CNN
        input_size - determines the size of the first input layer
        hidden_layers - comma separated string of hidden_layer sizes
        output_size - int of the number of outputs desired
        learning_rate - determines the learning rate for the optimizer
        drop_p - determines the dropout probability for the classifier(default- 0.5)
    Returns:
        model - Pretrained feature CNN with untrained classifier
        criterion - loss function to train on (torch.nn.NLLLoss())
        optimizer - optimizer for new, untrained classifier (torch.optim.Adam)
    """
    #Freeze feature parameters
    for param in model.parameters():
        param.requires_grad = False
    
    hidden_layers = hidden_layers.split(',')
    hidden_layers = [int(x) for x in hidden_layers]
    hidden_layers.append(output_size)
    
    #Take hidden_layer sizes and creates layer size definitions for each hidden_layer size combo
    layers = nn.ModuleList([nn.Linear(input_size,hidden_layers[0])])
    layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
    layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes])   
    
    net_layers = OrderedDict()
    
    #Creates hidden layers for each size passed by hidden_layers arg
    for x in range(len(layers)):
        layerid = x + 1
        if x == 0:
            net_layers.update({'drop{}'.format(layerid):nn.Dropout(p=drop_p)})
            net_layers.update({'fc{}'.format(layerid):layers[x]})
        else:
            net_layers.update({'relu{}'.format(layerid):nn.ReLU()})
            net_layers.update({'drop{}'.format(layerid):nn.Dropout(p=drop_p)})
            net_layers.update({'fc{}'.format(layerid):layers[x]})
        
    net_layers.update({'output':nn.LogSoftmax(dim=1)})
 
    #Define classifier
    classifier = nn.Sequential(net_layers)
    
    #Apply new classifier and generate criterion and optimizer
    model.classifier = classifier
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)
    
    return model, criterion, optimizer

def load_model(checkpoint):
    trained_model = torch.load('checkpoint.pth')
    arch = trained_model['arch']
    class_idx = trained_model['class_to_idx']
    #Only download the model you need, kill program if one of the three models isn't passed
    if arch == 'vgg':
        load_model = models.vgg16(pretrained=True)
    elif arch == 'alexnet':
        load_model = models.alexnet(pretrained=True)
    elif arch == 'densenet':
        load_model = models.densenet121(pretrained=True)
    else:
        print('{} architecture not recognized. Supported args: \'vgg\', \'alexnet\', or \'densenet\''.format(arch))
        sys.exit()
        
    for param in load_model.parameters():
        param.requires_grad = False
    
    load_model.classifier = trained_model['classifier']
    load_model.load_state_dict(trained_model['state_dict'])
    
    return load_model, arch, class_idx

if __name__ == '__main__':
    print('This is run as main.')